<a href="https://colab.research.google.com/github/srikarraju/NLP_Project/blob/main/Sentiment_Analysis_Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install -q ktrain

In [2]:
def get_sentence_labels(text):
  tweets = []
  labels = []
  lines = text.splitlines()
  for line in lines:
    print(line)
    words = line.split()
    labels.append(words[1])
    tweet_text = ''
    for i in range(2,len(words)):
      tweet_text += words[i]+' '
    tweets.append(tweet_text)
  return tweets,labels

In [3]:
#replace all emojis with phrases
import re
def replace_emojis(text):
  happyface = ":\)"
  text = re.sub(r"http.*","url",text)
  text = re.sub(happyface,"happyface",text)
  happyface = ":-\)"
  text = re.sub(happyface,"happyface",text)
  happyface = ":-\]"
  text = re.sub(happyface,"happyface",text)
  happyface = ":\]"
  text = re.sub(happyface,"happyface",text)
  happyface = "=\)"
  text = re.sub(happyface,"happyface",text)
  happyface = ":\'-\)"
  text = re.sub(happyface,"happyface",text)
  happyface = ":\'\)"
  text = re.sub(happyface,"happyface",text)
  sadface = ":\("
  text = re.sub(sadface,"sadface",text)
  sadface = ":-\("
  text = re.sub(sadface,"sadface",text)
  sadface = ":\["
  text = re.sub(sadface,"sadface",text)
  sadface = ":-\["
  text = re.sub(sadface,"sadface",text)
  sadface = ":\'-\("
  text = re.sub(sadface,"sadface",text)
  sadface = ":\'\("
  text = re.sub(sadface,"sadface",text)
  sadface = "\(:"
  text = re.sub(sadface,"sadface",text)
  # text = re.sub(":|","neutralface",text)
  text = re.sub(":P","lolface",text)
  text = re.sub(":p","lolface",text)
  text = re.sub(":o","surprise",text)
  text = re.sub(":O","surprise",text)
  #print(text[0:1000])
  return text

In [4]:
import re
def remove_punctuations(input_string):
  input_string = re.sub(r'\\u002c','',input_string)
  input_string = re.sub(r'@.*','',input_string)
  input_string = re.sub(r'#.*',' ',input_string)
  input_string = re.sub(r'[0-9]+', '', input_string)
  res = re.sub(r'[^\w\s]', '', input_string)
  return res

In [6]:
def remove_recurring_characters(sentences):
  def remove_recurring_characters_in_word(word):
    i = 1
    rec_count = 0
    while i < len(word):
      if word[i] == word[i-1]:
        rec_count += 1
      else:
        rec_count =0
      if rec_count>=2:
        word = word[:i]+word[i+1:]
        i -= 1
      i += 1
    return word
  for sent in sentences:
    for i in range(len(sent)):
      sent[i] = remove_recurring_characters_in_word(sent[i]).lower()
  print(sentences[0:10])
  return sentences

In [ ]:
train_file = open("/content/drive/MyDrive/Colab Notebooks/NLP/Project/datasets/train_2017",'r',encoding='utf-8')
text = train_file.read()
train_file.close()

text = replace_emojis(text)
text = remove_punctuations(text)

tweets,labels = get_sentence_labels2(text)

train_cnt = int(0.75*len(tweets))
train_x = tweets[0:train_cnt]
train_y = labels[0:train_cnt]
val_x = tweets[train_cnt:]
val_y = labels[train_cnt:]

class_names = ['positive','negative','neutral']

In [ ]:
import ktrain
from ktrain import text
MODEL_NAME = 'distilbert-base-uncased'
t = text.Transformer(MODEL_NAME, maxlen=500, classes=class_names)
trn = t.preprocess_train(train_x, train_y)
val = t.preprocess_test(val_x, val_y)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)

In [ ]:
learner.fit_onecycle(5e-5, 2)

In [9]:
def get_sentence_labels2(text):
  tweets = []
  labels = []
  lines = text.splitlines()
  for line in lines:
    print(line)
    words = line.split()
    labels.append(words[0])
    tweet_text = ''
    for i in range(1,len(words)):
      tweet_text += words[i]+' '
    tweets.append(tweet_text)
  return tweets,labels

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc=t)


test_file = open('/content/drive/MyDrive/Datasets/test_2016')
text2 = test_file.read()
test_file.close()

tweets,labels = get_sentences_labels(text2)

test_labels = []
for line in tweets:
  prediction = predictor.predict(line)
  test_labels.append(prediction)
#print(test_labels)

In [ ]:
def calculate_avg_recall(predictions,test_y):
  true_pos_predcited,total_true_pos = 0,0
  true_neg_predcited,total_true_neg = 0,0
  true_neu_predcited,total_true_neu = 0,0
  for i in range(len(predictions)):
    if test_y[i] == 2:
      total_true_pos += 1
      if predictions[i] == 2:
        true_pos_predcited += 1
    elif test_y[i] == 0:
      total_true_neg += 1
      if predictions[i] == 0:
        true_neg_predcited += 1
    elif test_y[i] == 1:
      total_true_neu += 1
      if predictions[i] == 1:
        true_neu_predcited += 1
  print(true_pos_predcited/total_true_pos,true_neg_predcited/total_true_neg,true_neu_predcited/total_true_neu)
  print(sum([true_pos_predcited/total_true_pos,true_neg_predcited/total_true_neg,true_neu_predcited/total_true_neu])/3)
  return sum([true_pos_predcited/total_true_pos,true_neg_predcited/total_true_neg,true_neu_predcited/total_true_neu])/3

In [ ]:
label_dict = {'neutral':1,'negative':0,'positive':2}
test_y,predictions = [],[]
for i in range(len(labels)):
  test_y.append(label_dict[labels[i]])
  predictions.append(label_dict[test_labels[i]])
calculate_avg_recall(predictions,test_y)

In [ ]:
import pandas as pd
output_df = pd.DataFrame({'predictions':predictions},index=None)
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/Project/Results/BERT_2016.csv',index=False) 